In [1]:
import pandas as pd
import censusdata
import os
import pandas as pd
import itertools
import argparse
import numpy as np 
from get_census_data import *

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2) 
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#if you don't have a census api key, go ahead and sign up for one
#here from the Census website for free:
#https://api.census.gov/data/key_signup.html  62718e387a10581e963b9ffaa5a506d22328267e
CENSUS_KEY = ENTER_YOUR_CENSUS_KEY_HERE

Here some useful functionalities of this code is intorduced. 
These functionalities directly or indirectly relate to the ACS data
and helps understanding the geographies or API definitions. 

# 1- Create US State FIPS Dictionary

This function generates a dictionary where keys are US State names and values 
are the state FIPS code. See this Wikipedia page to better understand FIPS code:

https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code

Once you run this for the first time the table contents are obtained from this source
and stored locally in the data folder in your local drive:

https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696

For every future run the dictionary is directly built from the local file and won't 
need an internet connection. 


In [4]:
#this will create a dictionary and store state_fips.csv inside data folder
fips_dict = create_state_fips_dict()
#check fips code for Texas
print('FIPS code for Texas is: %s'%fips_dict['Texas'])
print('FIPS code for Alabama is: %s'%fips_dict['Alabama'])

FIPS code for Texas is: 48
FIPS code for Alabama is: 01


# 2- Create an API Reference Table of 2019 ACS-5

This function generates a Pandas DataFrame from ACS-5 2019 APIs and their definitions.
Data is directly retreived from the below link in US Census website for the first time 
and is stored locally inside your data folder after the first run. 

https://api.census.gov/data/2019/acs/acs5/variables.html

For every future run it reads the data from local drive and won't need an internet connection. 

In [5]:
census_apis_df = create_data_api_table()
census_apis_df.head(5)

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANHH,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
1,AIHHTL,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
2,AIRES,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
3,ANRC,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
4,B01001_001E,Estimate!!Total:,SEX BY AGE,not required,"B01001_001EA, B01001_001M, B01001_001MA",0,int,B01001,NaN


Now you can use this table to figure the definition of the each API which will
be obtained from ACS5 data using the below line of code

In [11]:
#I would like to understand the definition of B19301_001E! 
print(census_apis_df[census_apis_df.Name=='B19301_001E'].Label.values[0])

Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)


# 3 (Fun Stuff!)- Get a Comprehensive Dataset for Your Desired Geographic Area

The below section shows you how to obtain a full dataset for a geographic level of 
interest. You can choose 'county','tract', or 'block group' for the geo_level parameter. 
The year parameter can be either of 2019 and 2018. The state_name can be specified to limit
the dataset to the desired US state. If not, the default will use 'All' meaning for all US states.

If you chose 'All' for tract or block group levels the function will iterate over all states and 
obtains the data using the Census API. The good news is that results of each iteration will be stored
in your local drive as a CSV file and if you face any crash in the middle of run (e.g., a server 
response issue) you can just simply re run the same line of code and the function will first look into
your local drive and gets what you already have (which is much faster!) instead of sending a duplicate 
request to the Census server. 

The GEOID column represnts the FIPS code for each geography. See below link for more details about the FIPS code for various levels of geography: 

https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

Below I show how you can use this function for various geographic levels!

In [3]:
#the default parameters gives you 2019 ACS5 data for all US Counties
county_2019_all_df=get_census_data(CENSUS_KEY)
county_2019_all_df.head(2)

,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_026E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E,B01001_038E,B01001_039E,B01001_040E,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B08301_021E,B08301_001E,B08301_010E,B19301_001E,B23025_001E,B23025_002E,B23025_005E,B29003_001E,B29003_002E,B15002_002E,B15002_015E,B15002_016E,B15002_017E,B15002_018E,B15002_019E,B15002_032E,B15002_033E,B15002_034E,B15002_035E,B27010_001E,B27010_017E,B27010_033E,B27010_050E,B27010_066E,B14002_001E,B14002_003E,B14002_027E,B28002_001E,B28002_012E,B28002_013E,B28003_001E,B28003_002E,B28003_003E,B28003_005E,B28001_003E,B28001_004E,B28001_006E,B28001_008E,B28001_011E,B28002_003E,B28002_006E,B28002_007E,B28002_010E,B19049_001E,B25010_001E,GEOID,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
0,21565,11469,621,621,636,417,254,156,226,438,812,718,800,703,768,825,843,344,423,249,308,530,402,185,190,10096,564,497,711,387,177,95,135,327,593,573,587,540,622,640,827,215,412,200,339,572,474,315,294,183,8633,67,23194,17630,9285,525,15441,2234,8100,564,183,46,13,7203,660,237,14,10,19948,277,475,882,10,20782,2411,2139,7737,130,1978,7737,5984,40,438,4607,582,913,41,1753,51,823,4166,150,46650,2.57,17051,5.50,5.18,6.25,3.73,4.84,55.69,18.82,2.12,0.78,23194.0,52.67,5.65,14.47,88.71,8.24,1.68,25.57,77.34,0.52,5.66
1,29003,14298,828,757,748,544,497,218,190,588,924,910,1053,925,953,931,1018,378,588,344,393,545,416,200,350,14705,725,718,784,503,467,187,187,689,1058,952,927,794,838,878,1124,353,545,380,455,642,455,294,750,599,12211,52,27546,24117,13168,738,19265,1588,9928,1239,388,83,90,10445,1583,587,55,34,25735,171,464,507,12,28152,3328,3488,10797,437,1990,10797,9363,51,1099,7669,769,972,123,1434,51,1174,6330,108,57308,2.31,17107,5.35,5.09,5.28,3.61,6.02,56.64,18.01,4.91,0.43,27546.0,54.60,5.60,8.24,80.08,4.48,4.05,18.43,86.72,0.47,10.18


As you can see above, the output dataframe includs both raw and calculated fields.
To exclude raw APIs and only keep the calculated fields, use below line of code.

In [7]:
county_2019_all_df = county_2019_all_df[[c for c in county_2019_all_df.columns \
                                         if not c.endswith('E')]]
county_2019_all_df.head(1)
county_2019_all_df.describe()

,GEOID,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
0,17051,5.5,5.18,6.25,3.73,4.84,55.69,18.82,2.12,0.78,23194.0,52.67,5.65,14.47,88.71,8.24,1.68,25.57,77.34,0.52,5.66


,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
count,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00,3220.00
mean,5.78,6.10,6.42,3.88,5.12,53.89,18.80,5.05,0.91,27660.94,57.99,5.54,13.89,78.03,9.55,3.44,21.00,84.95,0.57,10.25
std,1.27,1.24,1.22,0.69,2.57,3.32,4.62,3.21,3.05,7206.52,8.38,3.35,7.28,9.50,5.09,1.89,8.80,7.19,0.64,4.17
min,0.42,0.00,0.00,0.00,0.00,34.17,3.20,0.00,0.00,6118.00,15.37,0.00,1.87,22.44,0.00,0.00,2.49,27.19,0.00,1.69
25%,5.04,5.36,5.73,3.53,3.93,52.04,15.86,3.15,0.08,23410.00,52.86,3.64,9.41,74.07,5.75,2.31,14.96,81.62,0.24,7.40
50%,5.75,6.06,6.39,3.90,4.60,53.95,18.48,4.39,0.31,27118.00,58.89,4.98,12.39,80.38,8.58,3.04,19.67,86.10,0.44,9.48
75%,6.39,6.77,7.04,4.23,5.49,55.79,21.22,6.25,0.73,31097.25,64.10,6.58,16.36,84.59,12.01,4.11,25.51,89.64,0.75,12.23
max,21.43,13.35,16.46,9.61,39.57,77.27,56.71,34.11,61.24,76592.00,82.99,34.12,60.01,100.00,46.30,23.88,75.56,98.32,13.93,33.31


Now lets get 2018 ACS-5 data for tracts of Rhode Island! Use state_name = 'All' downloads
data for all US states iteratively. If it crashes for any reason (e.g., server connection), simply rerun it and it will quickly catch up where it stopped since data for every 
state is locally stored and retrieved for next runs. In general, the first run for every state may take a little time (e.g., a few minutes for Rhode Island). For every future run data will be retreived from your local disk from data folder which will be much faster as long as it is not  deleted.

In [10]:
tract_2019_RI_df=get_census_data(CENSUS_KEY,state_name = 'Rhode Island', 
                                 year = 2018, geo_level = 'tract')
tract_2019_RI_df = tract_2019_RI_df[[c for c in tract_2019_RI_df.columns \
                                         if not c.endswith('E')]]
tract_2019_RI_df.head(1)
tract_2019_RI_df.describe()

,GEOID,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
0,44003020903,5.09,7.72,10.95,6.8,2.76,52.0,14.68,8.95,1.81,69199.0,66.12,5.29,5.07,37.81,2.76,2.63,9.64,93.04,1.14,5.32


,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
count,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00,240.00
mean,5.10,5.32,5.52,3.51,6.32,57.47,16.76,4.00,2.60,34761.50,65.03,6.26,12.29,66.45,5.34,2.62,15.25,87.77,0.39,6.42
std,2.48,2.30,2.20,1.69,9.65,7.62,6.67,3.81,2.83,13115.18,7.20,3.51,9.43,17.57,4.05,2.02,8.11,6.50,0.79,4.14
min,0.00,0.00,0.00,0.00,0.13,4.21,0.86,0.00,0.00,5219.00,25.49,0.07,1.83,8.43,0.00,0.00,1.17,62.35,0.00,0.95
25%,3.29,3.84,4.16,2.38,2.93,54.22,12.28,1.59,0.63,26501.25,61.41,3.67,5.39,55.07,2.33,1.26,9.42,84.73,0.00,3.68
50%,4.86,5.20,5.50,3.33,4.33,58.23,16.33,3.16,1.65,34039.00,65.42,5.50,8.28,68.47,4.06,2.22,14.06,88.66,0.00,5.31
75%,6.41,6.66,6.98,4.70,6.06,61.66,20.77,5.00,3.97,41907.50,69.23,7.94,16.95,79.24,7.28,3.13,18.20,92.11,0.59,8.30
max,11.89,13.76,12.16,9.42,92.62,81.95,42.24,37.44,16.56,98562.00,93.93,17.60,46.58,95.75,22.56,11.09,49.62,98.83,6.58,25.84


Now lets get 2019 ACS-5 data for block groups of Vermont! 

In [12]:
bg_2019_VT_df=get_census_data(CENSUS_KEY,state_name = 'Vermont', 
                                 year = 2019, geo_level = 'block group')
bg_2019_VT_df = bg_2019_VT_df[[c for c in bg_2019_VT_df.columns \
                                         if not c.endswith('E')]]
bg_2019_VT_df.head(1)
bg_2019_VT_df.describe()

,GEOID,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
0,500219632001,4.52,6.33,6.79,9.35,1.81,56.71,14.48,3.66,0.0,44985.0,72.95,0.0,10.33,59.11,6.94,4.36,13.45,86.55,2.55,7.27


,pct_age_under_5,pct_age_5to9,pct_age_10to14,pct_age_15to17,pct_age_18to21,pct_age_22to64,pct_age_65&over,pct_work_remote,pct_use_pub_trans,income_per_capta,pct_labor_force,pct_unemployed,pct_income_below_poverty,pct_below_bsc,pct_no_insurance,pct_internet_no_subscr,pct_no_internet,pct_has_a_computer,pct_has_computer_w_diapup_subscr,pct_computer_no_internet_subscr
count,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00,521.00
mean,4.57,5.03,5.23,3.43,5.62,56.26,19.85,7.15,1.20,34597.59,65.11,3.69,10.59,62.78,4.12,5.09,13.31,89.80,0.71,8.95
std,2.94,2.93,2.90,2.31,9.10,8.55,7.57,5.24,2.71,10890.91,9.56,3.50,8.79,16.19,3.16,6.64,7.74,6.79,1.09,7.79
min,0.00,0.00,0.00,0.00,0.00,1.77,0.00,0.00,0.00,3541.00,23.70,0.00,0.00,13.71,0.00,0.00,0.00,56.13,0.00,0.00
25%,2.64,2.94,3.30,1.97,2.07,51.75,14.90,3.39,0.00,27484.00,59.35,1.23,5.12,51.48,1.79,1.08,7.81,86.21,0.00,3.70
50%,4.32,4.90,5.15,3.12,3.56,56.65,19.25,6.40,0.00,33198.00,65.73,2.99,8.61,65.35,3.84,3.11,13.20,90.48,0.00,7.27
75%,6.12,6.70,6.87,4.67,5.57,60.69,23.97,9.84,1.09,39287.00,71.70,4.94,13.09,74.47,5.66,6.92,17.74,94.61,1.29,12.32
max,19.33,17.97,16.28,17.93,95.84,85.02,59.80,29.31,20.71,95886.00,94.37,22.97,70.27,97.93,23.83,70.00,47.07,100.00,5.94,70.00
